# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')

cities_df.head()


,Unnamed: 0,City Name,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hofn,64.25,-15.21,54.32,85,100,13.65,IS,1593296014
1,1,Rikitea,-23.12,-134.97,70.54,75,0,17.16,PF,1593295847
2,2,Busselton,-33.65,115.33,60.01,75,100,22.15,AU,1593295847
3,3,Huilong,31.81,121.66,78.01,99,100,1.99,CN,1593296078
4,4,Idrinskoye,54.37,92.14,58.60,86,3,2.53,RU,1593296078


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Configure gmaps
gmaps.configure(api_key=g_key)

 # set coordinates for cities and use Humidity as the weight
city_coord = cities_df[['Lat', 'Lng']]
humidity_weight = cities_df['Humidity'].astype(float)

# create a figure
figure_layout = {
    'width': '1200px',
    'height': '550px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=(0,0), zoom_level=2.2, layout=figure_layout)

# add the heat layer
heat_layer = gmaps.heatmap_layer(city_coord, weights=humidity_weight, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

# render
fig


Figure(layout=FigureLayout(border='1px solid black', height='550px', padding='1px', width='1200px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [5]:
# Add marker layer ontop of heat map


# Display figure
